<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
import requests

# URL of the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform an HTTP GET request
response = requests.get(url)

# Assign the response to an object
html_content = response.text

# Display the status code and part of the HTML content for verification
print(response.status_code)
print(html_content[:500])


200
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clien


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
from bs4 import BeautifulSoup

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Output the title of the page
print(soup.title)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute

# Output the title of the page
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function to get all tables
html_tables = soup.find_all('table')

# Print the number of tables found
print(f"Number of tables found: {len(html_tables)}")
# Extract column names from the first table
if html_tables:
    # Select the first table
    first_table = html_tables[0]

    # Find all header cells
    headers = first_table.find_all('th')

    # Extract and print column names
    column_names = [header.get_text(strip=True) for header in headers]
    print("Column names:", column_names)
else:
    print("No tables found on the page.")


Number of tables found: 15
Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable" style="width:100%; font-size:1.00em; line-height:1.5em;">
<tbody><tr class="is-sticky">
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_26-2"><a href="#cite_note-booster-26">[c]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_27-2"><a href="#cite_note-Dragon-27">[d]</a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>
<tr>
<td rowspan="2">11 July 2024<br/>~02:00<sup class="reference" id="cite_ref-nextSFupcoming_379-1"><a href="#cite_note-nextSFupcoming-379">[371]</a></sup>
</td>
<td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a>
</td>
<td><a href="/wiki/Vandenberg_Space_Fo

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
# Define the function to extract column name from header
def extract_column_from_header(th):
    if th.a:  # If the header contains a link, extract the link text
        return th.a.get_text(strip=True)
    else:  # Otherwise, extract the header text
        return th.get_text(strip=True)

# Extract column names from the third table
column_names = []

if first_launch_table:
    # Find all 'th' elements in the header row
    header_row = first_launch_table.find('tr')
    th_elements = header_row.find_all('th')
    
    # Iterate through each 'th' element and extract the column name
    for th in th_elements:
        name = extract_column_from_header(th)
        if name is not None and len(name) > 0:
            column_names.append(name)

# Print the extracted column names
print("Column names:", column_names)

Column names: ['UTC', 'Version,booster', 'Launch site', '[d]', 'Orbit', 'Customer']


Check the extracted column names


In [12]:
print(column_names)

['UTC', 'Version,booster', 'Launch site', '[d]', 'Orbit', 'Customer']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform an HTTP GET request
response = requests.get(url)

# Assign the response to an object
html_content = response.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Use the find_all function to get all tables
html_tables = soup.find_all('table')

# Select the third table
if len(html_tables) >= 3:
    first_launch_table = html_tables[2]
else:
    print("The third table was not found.")
    first_launch_table = None

# Define the function to extract column name from header
def extract_column_from_header(th):
    if th.a:  # If the header contains a link, extract the link text
        return th.a.get_text(strip=True)
    else:  # Otherwise, extract the header text
        return th.get_text(strip=True)

# Extract column names from the third table
column_names = []

if first_launch_table:
    # Find all 'th' elements in the header row
    header_row = first_launch_table.find('tr')
    th_elements = header_row.find_all('th')
    
    # Iterate through each 'th' element and extract the column name
    for th in th_elements:
        name = extract_column_from_header(th)
        if name is not None and len(name) > 0:
            column_names.append(name)

# Print the extracted column names
print("Column names:", column_names)

# Initialize the dictionary with the extracted column names
launch_dict = dict.fromkeys(column_names)

# Initialize each value in the dictionary as an empty list
for key in launch_dict.keys():
    launch_dict[key] = []

# Add new columns with empty lists
additional_columns = ['Version Booster', 'Booster landing', 'Date', 'Time']
for col in additional_columns:
    launch_dict[col] = []

# Function to parse the date and time
def parse_date_time(date_time_str):
    date_time_parts = date_time_str.split()
    if len(date_time_parts) >= 2:
        date = date_time_parts[0]
        time = date_time_parts[1]
    else:
        date = date_time_parts[0]
        time = ''
    return date, time

# Iterate over each row in the table and populate the dictionary
rows = first_launch_table.find_all('tr')[1:]  # Skip the header row
for row in rows:
    cells = row.find_all('td')
    if len(cells) == len(column_names):
        for i, cell in enumerate(cells):
            text = cell.get_text(strip=True)
            if column_names[i] == 'Date and time':
                date, time = parse_date_time(text)
                launch_dict['Date'].append(date)
                launch_dict['Time'].append(time)
            else:
                launch_dict[column_names[i]].append(text)

# Ensure all lists in the dictionary are of the same length
max_length = max(len(lst) for lst in launch_dict.values())
for key, lst in launch_dict.items():
    while len(lst) < max_length:
        lst.append(None)

# Convert the dictionary to a Pandas DataFrame
launch_df = pd.DataFrame(launch_dict)

# Display the DataFrame
print(launch_df.head())


Column names: ['UTC', 'Version,booster', 'Launch site', '[d]', 'Orbit', 'Customer']
                        UTC   Version,booster   Launch site  \
0   11 July 2024~02:00[371]             F9 B5   VSFB,SLC-4E   
1   13 July 2024~07:58[371]             F9 B5  CCSFS,SLC-40   
2         15 July 2024[372]  F9 B5♺[333][373]  CCSFS,SLC-40   
3         18 July 2024[371]             F9 B5   VSFB,SLC-4E   
4  Mid? July 2024[377][378]             F9 B5   VSFB,SLC-4E   

                                                 [d] Orbit  \
0                 Starlink Group 9–3(~20 satellites)   LEO   
1                Starlink Group 10–4(~23 satellites)   LEO   
2  WorldView Legion 3–4Mission 2 (2 sats)[374][37...   SSO   
3            Transporter-11, SmallSat Rideshare[299]   SSO   
4                   ASBM 1(GX10A),[379]ASBM 2(GX10B)   HEO   

                Customer Version Booster Booster landing  Date  Time  
0                 SpaceX            None            None  None  None  
1                 Spac

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform an HTTP GET request
response = requests.get(url)

# Assign the response to an object
html_content = response.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Use the find_all function to get all tables
html_tables = soup.find_all('table')

# Select the third table
if len(html_tables) >= 3:
    first_launch_table = html_tables[2]
else:
    print("The third table was not found.")
    first_launch_table = None

# Define the function to extract column name from header
def extract_column_from_header(th):
    if th.a:  # If the header contains a link, extract the link text
        return th.a.get_text(strip=True)
    else:  # Otherwise, extract the header text
        return th.get_text(strip=True)

# Extract column names from the third table
column_names = []

if first_launch_table:
    # Find all 'th' elements in the header row
    header_row = first_launch_table.find('tr')
    th_elements = header_row.find_all('th')
    
    # Iterate through each 'th' element and extract the column name
    for th in th_elements:
        name = extract_column_from_header(th)
        if name is not None and len(name) > 0:
            column_names.append(name)

# Print the extracted column names
print("Column names:", column_names)

# Initialize the dictionary with the extracted column names
launch_dict = dict.fromkeys(column_names)

# Initialize each value in the dictionary as an empty list
for key in launch_dict.keys():
    launch_dict[key] = []

# Add new columns with empty lists
additional_columns = ['Version Booster', 'Booster landing', 'Date', 'Time']
for col in additional_columns:
    launch_dict[col] = []

# Function to parse the date and time
def parse_date_time(date_time_str):
    date_time_parts = date_time_str.split()
    if len(date_time_parts) >= 2:
        date = date_time_parts[0]
        time = date_time_parts[1]
    else:
        date = date_time_parts[0]
        time = ''
    return date, time

# Iterate over each row in the table and populate the dictionary
rows = first_launch_table.find_all('tr')[1:]  # Skip the header row
for row in rows:
    cells = row.find_all('td')
    if len(cells) == len(column_names):
        for i, cell in enumerate(cells):
            # Remove unwanted annotations and reference links
            text = cell.get_text(strip=True).split('[')[0].replace('N/A', '').strip()
            if column_names[i] == 'Date and time':
                date, time = parse_date_time(text)
                launch_dict['Date'].append(date)
                launch_dict['Time'].append(time)
            else:
                launch_dict[column_names[i]].append(text)

# Ensure all lists in the dictionary are of the same length
max_length = max(len(lst) for lst in launch_dict.values())
for key, lst in launch_dict.items():
    while len(lst) < max_length:
        lst.append(None)

# Convert the dictionary to a Pandas DataFrame
launch_df = pd.DataFrame(launch_dict)

# Display the DataFrame
print(launch_df.head())


Column names: ['UTC', 'Version,booster', 'Launch site', '[d]', 'Orbit', 'Customer']
                  UTC Version,booster   Launch site  \
0  11 July 2024~02:00           F9 B5   VSFB,SLC-4E   
1  13 July 2024~07:58           F9 B5  CCSFS,SLC-40   
2        15 July 2024          F9 B5♺  CCSFS,SLC-40   
3        18 July 2024           F9 B5   VSFB,SLC-4E   
4      Mid? July 2024           F9 B5   VSFB,SLC-4E   

                                      [d] Orbit               Customer  \
0      Starlink Group 9–3(~20 satellites)   LEO                 SpaceX   
1     Starlink Group 10–4(~23 satellites)   LEO                 SpaceX   
2  WorldView Legion 3–4Mission 2 (2 sats)   SSO     Maxar Technologies   
3      Transporter-11, SmallSat Rideshare   SSO                Various   
4                          ASBM 1(GX10A),   HEO  Space Norway/Inmarsat   

  Version Booster Booster landing  Date  Time  
0            None            None  None  None  
1            None            None  None  Non

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [15]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [16]:
import requests
import pandas as pd

# URL of the SpaceX API for launches
url = "https://api.spacexdata.com/v4/launches"

# Perform an HTTP GET request to fetch the data
response = requests.get(url)

# Convert the response to JSON
launches_data = response.json()

# Convert the JSON data to a Pandas DataFrame
df = pd.json_normalize(launches_data)

# Display the first few rows of the dataframe to understand its structure
print(df.head())

# Filter out Falcon 1 launches
falcon_9_launches = df[df['rocket'] != '5e9d0d95eda69955f709d1eb']  # The rocket ID for Falcon 1

# Get the number of Falcon 9 launches
number_of_falcon_9_launches = falcon_9_launches.shape[0]

print("Number of Falcon 9 launches:", number_of_falcon_9_launches)


       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False     0.0   
4                      None                    NaN  False     0.0   

                     rocket success  \
0  5e9d0d95eda69955f709d1eb   False   
1  5e9d0d95eda69955f709d1eb   False   
2  5e9d0d95eda69955f709d1eb   False   
3  5e9d0d95eda69955f709d1eb    True   
4  5e9d0d95eda69955f709d1eb    True   

                                            failures  \
0  [{'time': 33, 'altitude': None, 'reason': 'mer...   
1  [{'time': 301, 'altitude': 289, 'reason': 'har...   
2  [{'time': 140, 'altitude': 35, 'reason': 'resi...   
3                                                 []   
4                                                 []   

             

In [17]:
import requests
import pandas as pd

# URL of the SpaceX API for launches
url = "https://api.spacexdata.com/v4/launches"

# Perform an HTTP GET request to fetch the data
response = requests.get(url)

# Convert the response to JSON
launches_data = response.json()

# Convert the JSON data to a Pandas DataFrame
df = pd.json_normalize(launches_data)

# Display the column names to understand the DataFrame structure
print(df.columns)

# Inspect a few rows to find the column related to landing pad
print(df.head())

# Count the missing values in the identified landing pad column
# Let's assume the correct column name is 'cores[0].landpad' based on the data inspection
missing_values_count = df['cores[0].landpad'].isna().sum()

print("Number of missing values in the 'cores[0].landpad' column:", missing_values_count)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2 

KeyError: 'cores[0].landpad'

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
